In [1]:
#import packages
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape, Point
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request, json
import requests
import urllib.parse
import datetime
import os
import dask.dataframe as dd
import calendar
from datetime import datetime, timedelta

from geopandas.tools import sjoin

import warnings
warnings.filterwarnings('ignore')

/Users/lizjohnson/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
comp_dict_final = {'noise': ['noise - residential','noise survey', 'noise - commercial', 'noise - helicopter', 'collection truck noise', 'noise - vehicle', 'noise - house of worship', 'noise', 'noise - park', 'noise - street/sidewalk'],
                   'taxi complaints & inquiries': ['green taxi complaint', 'green taxi report', 'taxi compliment', 'dispatched taxi compliment', 'dispatched taxi complaint', 'for hire vehicle complaint', 'taxi complaint', 'taxi licensee complaint', 'taxi report', 'fhv licensee complaint', 'for hire vehicle report'],
                   'homelessness': ['homeless person assistance', 'homeless street condition', 'encampment', 'homeless encampment'],
                   'plaster/paint': ['paint/plaster','peeling paint', 'paint - plaster'],
                   'street/sidewalk/road condition': ['street condition', 'dep street condition', 'street light condition', 'root/sewer/sidewalk condition', 'dep sidewalk condition', 'dep bridge condition', 'curb condition', 'lot condition'],
                   'animal abuse': ['animal abuse', 'animal-abuse'],
                   'dead or damaged tree': ['dead tree', 'damaged tree', 'dead/dying tree'],
                   'waste & bulky item collection': ['institution disposal complaint', 'missed collection', 'missed collection (all materials)', 'residential disposal complaint', 'electronics waste appointment', 'seasonal collection', 'request large bulky item collection', 'request xmas tree collection', 'electronics waste'],
                   'litter basket complaints & inquiries': ['overflowing litter baskets', 'litter basket request', 'litter basket / request', 'litter basket complaint'],
                   'recycling basket complaints': ['recycling basket complaint', 'overflowing recycling baskets'],
                   'literature request': ['literature request', 'oem literature request', 'dor literature request', 'dof literature request', 'hpd literature request'], 
                   'hazardous materials': ['radioactive material', 'hazardous material', 'hazardous materials'],
                   'broken parking meter': ['broken parking meter', 'broken muni meter'],
                   'street sign complaints': ['highway sign - missing', 'highway sign - damaged', 'highway sign - dangling', 'street sign - dangling', 'street sign - missing', 'street sign - damaged'], 
                   'derelict vehicles': ['derelict vehicle', 'derelict vehicles'], 
                   'bus stop shelter complaint': ['bus stop shelter complaint', 'bus stop shelter placement'],
                   'illegal animal - sold/kept': ['illegal animal - sold/kept', 'illegal animal sold', 'illegal animal kept as pet'],
                   'home delivered meal complaint': ['home delivered meal - missed delivery', 'home delivered meal complaint'],
                   "building marshal's office": ["building marshal's office", 'building marshals office'],
                   'street sweeping complaint': ['sweeping/missed', 'sweeping/missed-inadequate', 'sweeping/inadequate'],
                   'fire alarm serive requests': ['fire alarm - new system', 'fire alarm - modification', 'fire alarm - reinspection', 'fire alarm - replacement'],
                   'general inquiry': ['general question', 'comments', 'general'],
                   'dof property complaints & inquiries': ['dof property - owner issue', 'dof property - update account', 'dof property - rpie issue', 'dof property - city rebate', 'dof property - reduction issue', 'dof property - payment issue', 'dof property - property value', 'dof property - request copy', 'dof property - state rebate'], 
                   'public advocate request': ['advocate - other', 'advocate-prop class incorrect', 'advocate-scrie/drie', 'advocate-prop refunds/credits', 'advocate-business tax', 'advocate-co-opcondo abatement', 'advocate-foreclosure', 'advocate - rpie', 'advocate - levy', 'advocate-commercial exemptions', 'advocate-personal exemptions', 'advocate-ubt', 'advocate - lien', 'advocate-property value'],
                   'dof parking requests': ['dof parking - payment issue', 'dof parking - request status', 'dof parking - request copy', 'dof parking - dmv clearance', 'dof parking - address update', 'dof parking - tax exemption'],
                   'public assembly': ['public assembly - temporary', 'public assembly'], 
                   'dhs advantage program inquiries': ['dhs advantage - tenant', 'dhs advantage - third party', 'dhs advantage -landlord/broker'],
                   'construction': ['general construction', 'construction'],
                   'electrical': ['electric', 'electrical'],
                   'unsanitary condition': ['unsanitary condition'],
                   'asbestos': ['asbestos', 'asbestos/garbage nuisance'],
                   'dirty conditions': ['dirty condition', 'dirty conditions'],
                  'snow' : ['snow','snw','snow or ice','snow removal'],
                  'ferry complaints & inquiries':['ferry inquiry','ferry permit','ferry complaint'],
                  'building facade complaints & inquiries':['facade insp safety pgm','facades']}

In [3]:
all311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
uniqueid = 'unique_key'
cat_clnm = 'agency_name'
subcat_clnm = 'complaint_type'
subsubcat_clnm = 'descriptor'
datetime_clnm = 'created_date'

In [43]:
url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
query = (url +'?'
        "$select=agency_name,count(unique_key)"
        "&$where=created_date between '2010-01-01T00:00:00.000' and '2022-12-31T23:59:59.999'"
        "&$group=agency_name"
        "&$limit=1000000")
query = query.replace(" ", "%20")
print(query)
response = urllib.request.urlopen(query)
data = json.loads(response.read())

https://data.cityofnewyork.us/resource/erm2-nwe9.json?$select=agency_name,count(unique_key)&$where=created_date%20between%20'2010-01-01T00:00:00.000'%20and%20'2022-12-31T23:59:59.999'&$group=agency_name&$limit=1000000


In [111]:
df = pd.DataFrame(data)
df['count_unique_key'] = df['count_unique_key'].astype(int)
df.sort_values(by = 'count_unique_key',ascending = False).head(10)

,agency_name,count_unique_key
69,New York City Police Department,9235192
28,Department of Housing Preservation and Develop...,7629926
32,Department of Transportation,3730438
31,Department of Sanitation,3704995
24,Department of Environmental Protection,2211927
21,Department of Buildings,1440895
30,Department of Parks and Recreation,1341426
26,Department of Health and Mental Hygiene,783671
1881,Taxi and Limousine Commission,322335
22,Department of Consumer Affairs,274090


In [112]:
df.sort_values(by = 'count_unique_key',ascending = False).tail(10)

,agency_name,count_unique_key
1655,School - Rosalyn Yalow Charter School,1
832,School - Peace and Diversity Academy,1
1653,School - Rockaway Park High School for Environ...,1
1652,School - Rockaway Collegiate High School,1
843,School - Professional Performing Arts,1
1355,School - PS 41 Crocheron,1
839,School - Performing Arts and Technology High S...,1
1648,School - Riverton Street Charter School,1
1646,School - Riverdale - Kingsbridge Academy (Midd...,1
227,School - Brooklyn Democracy Academy,1


In [113]:
agency_dict = {'NYC Public Schools':df[df['agency_name'].str.contains('School')]['agency_name'].tolist()}

In [114]:
print(agency_dict)

{'NYC Public Schools': ['School - 47 The American Sign Language and English Secondary School', 'School - 51 Avenue Academy: The Path To Academic Excellence', 'School - Abraham Lincoln High School', 'School - Academic Leadership Charter School', 'School - Academy for College Preparation and Career Exploration: A College Board School', 'School - Academy for Environmental Leadership', 'School - Academy for Health Careers', 'School - Academy for Language and Technology', 'School - Academy for Personal Leadership and Excellence', 'School - Academy for Public Relations', 'School - Academy for Scholarship and Entrepreneurship', 'School - Academy for Scholarship and Entrepreneurship: A College Board School', 'School - Academy for Social Action', 'School - Academy for Software Engineering', 'School - Academy for Young Writers A', 'School - Academy for Young Writers B', 'School - Academy of American Studies', 'School - Academy of Applied Mathematics and Technology', 'School - Academy of Arts and

In [115]:
mask = df['agency_name'].str.contains('School')

# replace the selected rows with 'NYC Public Schools'
df.loc[mask, 'agency_name'] = 'NYC Public Schools'

In [116]:
df = df.groupby('agency_name').sum()
df.sort_values(by = 'count_unique_key',ascending = False).head(10)

,count_unique_key
agency_name,
New York City Police Department,9235192
Department of Housing Preservation and Development,7629926
Department of Transportation,3730438
Department of Sanitation,3704995
Department of Environmental Protection,2211927
Department of Buildings,1440895
Department of Parks and Recreation,1341426
Department of Health and Mental Hygiene,783671
Taxi and Limousine Commission,322335


In [117]:
df.sort_values(by = 'count_unique_key',ascending = False).tail(10)

,count_unique_key
agency_name,
MOC,3
NYCEM,2
Alternative Superintendency,2
OMB,2
Fire Department Operations Center,2
DOF,2
Bureau of Tobacco Control,1
New York Police Precinct,1
Early Care and Education Information Unit,1


In [118]:
len(df)

96

In [119]:
from fuzzywuzzy import fuzz, process

# list of unique agency names
agency_list = df.index.tolist()

# create a dictionary to store the grouped agency names
grouped_agencies = {}

# loop through each agency name and compare it with the others using fuzz.token_set_ratio
for i, agency in enumerate(agency_list):
    # create a list of tuples containing the agency name and its token set ratio score
    ratios = process.extract(agency, agency_list, scorer=fuzz.token_set_ratio)
    
    # filter out the agency itself and any duplicates
    ratios = [x for x in ratios if x[0] != agency and x[0] not in grouped_agencies]
    
    # create a list of agency names that have a token set ratio score >= 80
    similar_agencies = [x[0] for x in ratios if x[1] >= 80]
    
    # add the agency and its similar agencies to the grouped_agencies dictionary
    grouped_agencies[agency] = [agency] + similar_agencies

In [120]:
# print the grouped agencies
grouped_agencies

{'3-1-1': ['3-1-1', '3-1-1 Call Center'],
 '3-1-1 Call Center': ['3-1-1 Call Center'],
 '311 Administrative Supervisor': ['311 Administrative Supervisor'],
 'ACS': ['ACS'],
 'Accounts Examination': ['Accounts Examination'],
 'Adjudication - Appeals Unit': ['Adjudication - Appeals Unit'],
 'Adjudication - Hearing by Mail': ['Adjudication - Hearing by Mail'],
 'Alternative Superintendency': ['Alternative Superintendency'],
 'Bureau of Tobacco Control': ['Bureau of Tobacco Control'],
 'CEO': ['CEO'],
 'COIB': ['COIB'],
 'Central - Department of Education': ['Central - Department of Education',
  'Department of Education',
  'Office of the CIO - Department of Education'],
 'Child Care and Camps Complaint Unit': ['Child Care and Camps Complaint Unit'],
 'Commercial Exemption Unit': ['Commercial Exemption Unit',
  'Exemption Unit',
  'Exemption Policy Unit'],
 'Condo or CoOp Unit': ['Condo or CoOp Unit'],
 'Correspondence - Taxi and Limousine Commission': ['Correspondence - Taxi and Limousin

In [121]:
agency_dict['3-1-1 Call Center'] = ['3-1-1','311 Administrative Supervisor']
agency_dict['Department of Education'] = ['DOE','Central - Department of Education', 'Department of Education','Office of the CIO - Department of Education']
agency_dict['Taxi and Limousine Commission'] = ['Correspondence - Taxi and Limousine Commission','TLC']
agency_dict['Department of Buildings'] = ['DOB','DOB Inspections - Bronx','DOB Inspections - Brooklyn','DOB Inspections - Manhattan','DOB Inspections - Queens']                                           
agency_dict['Department of Finance'] = ['DOF', 'DOF Legal Affairs - Taxpayer Services Unit']
agency_dict['Department of Homeless Services'] = 'Operations Unit - Department of Homeless Services'
agency_dict['Exemption Policy Unit'] = ['Exemption Unit','Personal Exemption Unit']
agency_dict['Fire Department of New York'] = 'Fire Department Operations Center'
agency_dict['New York City Police Department'] = 'New York Police Precinct'
agency_dict['NYC Emergency Management'] = ['NYCEM', 'NYCEM Property Damage']

In [122]:
df = df.reset_index()
for i, row in df.iterrows():
    for key, value in agency_dict.items():
        if row['agency_name'] in value:
            df.loc[i, 'agency_name'] = key

In [123]:
df.head()

,agency_name,count_unique_key
0,3-1-1 Call Center,14
1,3-1-1 Call Center,20577
2,3-1-1 Call Center,3
3,ACS,1219
4,Accounts Examination,10754


In [124]:
df = df.groupby('agency_name').sum()
df.sort_values(by = 'count_unique_key',ascending = False).tail(10)

,count_unique_key
agency_name,
HPD,6
CEO,5
Tax Lien Unit,5
DCA,4
MOC,3
DFTA,3
Alternative Superintendency,2
OMB,2
Early Care and Education Information Unit,1


In [125]:
df.sort_values(by = 'count_unique_key',ascending = False).head(10)

,count_unique_key
agency_name,
New York City Police Department,9235193
Department of Housing Preservation and Development,7629926
Department of Transportation,3730438
Department of Sanitation,3704995
Department of Environmental Protection,2211927
Department of Buildings,1469191
Department of Parks and Recreation,1341426
Department of Health and Mental Hygiene,783671
Taxi and Limousine Commission,327924


In [126]:
df = df[df['count_unique_key']>100]
print(len(df))

54


In [127]:
df.sort_values(by = 'count_unique_key',ascending = False).tail(10)

,count_unique_key
agency_name,
Traffic Management Center,1723
Mosquito Control,1715
Child Care and Camps Complaint Unit,1353
ACS,1219
Division of Alternative Management,561
TAX,461
Office of the Sheriff,228
Department of Finance,202
Office of Technology and Innovation,176


In [129]:
agency_list = df.index.unique().tolist()
print(len(agency_list))

54


In [143]:
def get_complaints_by_month(date_str, nta_nyc):
    year_month = datetime.strptime(date_str, '%Y-%m')
    next_month = year_month.replace(day=28) + timedelta(days=4)  # adds 4 days to get to the next month
    last_day = (next_month - timedelta(days=next_month.day)).strftime('%Y-%m-%d')
    url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
    query = (url +'?'
            "$select= unique_key,agency_name,latitude,longitude,created_date"
            "&$where=created_date between '"+date_str+'-01T00:00:00.000'+"' and '"+last_day+'T23:59:59.999'+"'"
            "&$limit=1000000")
    query = query.replace(" ", "%20")
    # Retry the query up to 5 times if it fails
    for i in range(5):
        try:
            response = urllib.request.urlopen(query)
            data = json.loads(response.read())
            
            df = pd.DataFrame(data)[['agency_name', 'latitude', 'longitude', 'created_date', 'unique_key']]
            df = df[df['latitude'].notnull()]
            df = df[df['longitude'].notnull()]
            # Step 2: Assign complaints to neighborhoods
            geometry = [Point(xy) for xy in zip(df.longitude.astype(float), df.latitude.astype(float))]
            crs = {'init': 'epsg:4326'}
            complaints = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
            complaints = gpd.sjoin(complaints, nta_nyc, how='inner',predicate = 'within')

            # Step 3: Clean up complaint types
            for complaint_type in complaints['agency_name'].unique():
                stripped_complaint_type = complaint_type.strip().lower()

                if stripped_complaint_type in [val_item.strip().lower() for val in agency_dict.values() for val_item in val]:
                    key = [k for k, v in agency_dict.items() if stripped_complaint_type in [val_item.strip().lower() for val_item in v]][0]
                    complaints.loc[complaints['agency_name'] == complaint_type, 'agency_name'] = key
            
            complaints = complaints[complaints['agency_name'].isin(agency_list)]
            # Step 4: Aggregate complaints by neighborhood and complaint type
            complaints = complaints.groupby(['ntacode', 'agency_name']).agg({'unique_key': 'nunique'})
            complaints = complaints.unstack(level='agency_name')
            complaints.columns = complaints.columns.droplevel()
            complaints.index.name = 'ntacode'
            complaints.reset_index(inplace=True)
            complaints['date'] = pd.to_datetime(date_str)
            complaints.set_index(['ntacode', 'date'], inplace=True)
        
        except Exception as e:
            print(f"Error fetching complaints for {date_str}: {e}")
            if i < 4:
                print(f"Retrying in {5 * (i+1)} seconds...")
                #time.sleep(5 * (i+1))
            elif i ==4: 
                print(f"Unable to load data for {date_str} even after 5 attempts. Returning NaN values for all NTACodes.")
                index = pd.MultiIndex.from_product([nta_nyc.ntacode.unique().tolist(), comp_dict_final.keys()], names=['ntacode', 'agency_name'])
                df = pd.DataFrame({'count': np.nan}, index=index)
                complaints = df.unstack(level='agency_name')
                complaints.columns = complaints.columns.droplevel()
                complaints.index.name = 'ntacode'
                complaints.reset_index(inplace=True)
                complaints['date'] = pd.to_datetime(date_str)
                complaints.set_index(['ntacode', 'date'], inplace=True)
    complaints = complaints.fillna(0)
    return complaints

In [131]:
#import nta shapefile
nta_url = 'https://data.cityofnewyork.us/resource/q2z5-ai38.json'

query = (nta_url+'?'
        "$select=*"
        "&$limit=10000")
query = query.replace(" ", "%20")
response = urllib.request.urlopen(query)
data = json.loads(response.read())


for d in data:
    d['geometry'] = shape(d['the_geom'])

#store in a geodataframe
nta_nyc = gpd.GeoDataFrame(data, geometry = 'geometry', crs = 'EPSG: 4326')
nta_nyc = nta_nyc.drop(columns = ['the_geom'])
nta_nyc.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ..."
1,2,Bronx,005,BX28,Van Cortlandt Village,21945.719299,25666124.5948,"MULTIPOLYGON (((-73.88705 40.88435, -73.88705 ..."
2,4,Queens,081,QN55,South Ozone Park,36708.1693055,82461393.7368,"MULTIPOLYGON (((-73.80577 40.68293, -73.80552 ..."
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.6720656,14041667.99,"MULTIPOLYGON (((-73.98017 40.66115, -73.98021 ..."
4,3,Brooklyn,047,BK50,Canarsie,43703.6096661,82089678.6389,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ..."


In [144]:
# Step 5: Loop over all months and combine results
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2010-03-01')
complaints = pd.DataFrame()
for date in pd.date_range(start=start_date, end=end_date, freq='MS'):
    date_str = date.strftime('%Y-%m')
    print(date_str)
    monthly_complaints = get_complaints_by_month(date_str, nta_nyc)
    complaints = pd.concat([complaints, monthly_complaints])

2010-01
2010-02
2010-03


In [146]:
complaints.head(10)

,agency_name,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,Department of Housing Preservation and Development,Department of Information Technology and Telecommunications,Department of Parks and Recreation,Department of Sanitation,Department of Transportation,NYC Public Schools,New York City Police Department,Refunds and Adjustments,Taxi and Limousine Commission
ntacode,date,,,,,,,,,,,,,,,,,
BK09,2010-01-01,1.0,0.0,0.0,23.0,2.0,0.0,34.0,9.0,56.0,7.0,2.0,14.0,56.0,0.0,33.0,0.0,6.0
BK17,2010-01-01,0.0,0.0,0.0,78.0,10.0,2.0,127.0,21.0,258.0,1.0,24.0,91.0,288.0,0.0,156.0,0.0,0.0
BK19,2010-01-01,0.0,0.0,0.0,45.0,8.0,2.0,72.0,15.0,244.0,0.0,9.0,59.0,81.0,3.0,53.0,0.0,3.0
BK21,2010-01-01,0.0,0.0,1.0,31.0,1.0,0.0,64.0,9.0,261.0,0.0,7.0,27.0,59.0,0.0,79.0,0.0,2.0
BK23,2010-01-01,0.0,0.0,0.0,6.0,5.0,1.0,6.0,5.0,29.0,0.0,0.0,4.0,8.0,0.0,15.0,0.0,0.0
BK25,2010-01-01,0.0,0.0,0.0,31.0,6.0,0.0,95.0,15.0,305.0,1.0,13.0,68.0,123.0,0.0,115.0,0.0,1.0
BK26,2010-01-01,0.0,0.0,0.0,30.0,2.0,0.0,64.0,8.0,100.0,1.0,9.0,36.0,73.0,0.0,50.0,0.0,1.0
BK27,2010-01-01,0.0,0.0,0.0,30.0,2.0,0.0,26.0,8.0,149.0,0.0,7.0,30.0,65.0,1.0,65.0,0.0,0.0
BK28,2010-01-01,0.0,0.0,0.0,91.0,14.0,2.0,61.0,28.0,523.0,0.0,26.0,112.0,232.0,0.0,212.0,0.0,5.0


In [154]:
# Step 5: Loop over all months and combine results
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2022-12-01')
complaints = pd.DataFrame()
for date in pd.date_range(start=start_date, end=end_date, freq='MS'):
    date_str = date.strftime('%Y-%m')
    print(date_str)
    monthly_complaints = get_complaints_by_month(date_str, nta_nyc)
    complaints = pd.concat([complaints, monthly_complaints])

2010-01
2010-02
2010-03
2010-04
2010-05
2010-06
2010-07
2010-08
2010-09
2010-10
2010-11
2010-12
2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2011-12
2012-01
2012-02
2012-03
2012-04
2012-05
2012-06
2012-07
2012-08
2012-09
2012-10
2012-11
2012-12
2013-01
2013-02
2013-03
2013-04
2013-05
2013-06
2013-07
2013-08
2013-09
2013-10
2013-11
2013-12
2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
Error fetching complaints for 2014-10: Invalid control character at: line 49808 column 23 (char 8863163)
Retrying in 5 seconds...
2014-11
2014-12
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
2018-01
2018-02
2018-03
2018-04
2018-05
2018-06
2018-07
2018-08
2018-09
2018-10
2018-11
2018-12
2019-0

In [157]:
complaints.head()

,agency_name,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,Department of Housing Preservation and Development,Department of Information Technology and Telecommunications,...,Disability Rent Increase Exemption Unit,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation
ntacode,date,,,,,,,,,,,,,,,,,,,,,
BK09,2010-01-01,1.0,0.0,0.0,23.0,2.0,0.0,34.0,9.0,56.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK17,2010-01-01,0.0,0.0,0.0,78.0,10.0,2.0,127.0,21.0,258.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK19,2010-01-01,0.0,0.0,0.0,45.0,8.0,2.0,72.0,15.0,244.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK21,2010-01-01,0.0,0.0,1.0,31.0,1.0,0.0,64.0,9.0,261.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BK23,2010-01-01,0.0,0.0,0.0,6.0,5.0,1.0,6.0,5.0,29.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
complaints.tail()

,agency_name,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,Department of Housing Preservation and Development,Department of Information Technology and Telecommunications,...,Disability Rent Increase Exemption Unit,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation
ntacode,date,,,,,,,,,,,,,,,,,,,,,
SI37,2022-12-01,NaN,NaN,NaN,35.0,2.0,0.0,66.0,20.0,210.0,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
SI45,2022-12-01,NaN,NaN,NaN,18.0,1.0,1.0,72.0,11.0,31.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
SI48,2022-12-01,NaN,NaN,NaN,16.0,2.0,0.0,40.0,3.0,9.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
SI54,2022-12-01,NaN,NaN,NaN,53.0,2.0,0.0,108.0,11.0,29.0,NaN,...,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,0.0
SI99,2022-12-01,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0


In [159]:
complaints = complaints.fillna(0)
complaints.head()

,agency_name,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,Department of Housing Preservation and Development,Department of Information Technology and Telecommunications,...,Disability Rent Increase Exemption Unit,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation
ntacode,date,,,,,,,,,,,,,,,,,,,,,
BK09,2010-01-01,1.0,0.0,0.0,23.0,2.0,0.0,34.0,9.0,56.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK17,2010-01-01,0.0,0.0,0.0,78.0,10.0,2.0,127.0,21.0,258.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK19,2010-01-01,0.0,0.0,0.0,45.0,8.0,2.0,72.0,15.0,244.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK21,2010-01-01,0.0,0.0,1.0,31.0,1.0,0.0,64.0,9.0,261.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BK23,2010-01-01,0.0,0.0,0.0,6.0,5.0,1.0,6.0,5.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
complaints.columns

Index(['3-1-1 Call Center', 'Correspondence Unit', 'Department for the Aging',
       'Department of Buildings', 'Department of Consumer Affairs',
       'Department of Education', 'Department of Environmental Protection',
       'Department of Health and Mental Hygiene',
       'Department of Housing Preservation and Development',
       'Department of Information Technology and Telecommunications',
       'Department of Parks and Recreation', 'Department of Sanitation',
       'Department of Transportation', 'NYC Public Schools',
       'New York City Police Department', 'Refunds and Adjustments',
       'Taxi and Limousine Commission', 'Economic Development Corporation',
       'Division of Alternative Management', 'HRA Benefit Card Replacement',
       'Child Care and Camps Complaint Unit', 'NYC Emergency Management',
       'Mosquito Control', 'Office of the Taxpayer Advocate',
       'Disability Rent Increase Exemption Unit', 'Commercial Exemption Unit',
       'DRIE Unit', 'Traf

In [161]:
len(complaints.columns.tolist())

34

In [162]:
comp_cats = complaints.columns.tolist()
compset = set(comp_cats)
agencyset = set(agency_list)
agencyset-compset

{'ACS',
 'Accounts Examination',
 'Adjudication - Appeals Unit',
 'Adjudication - Hearing by Mail',
 'Condo or CoOp Unit',
 'DCAS',
 'DHS Advantage Programs',
 'DVS',
 'Department of Finance',
 'Dunning Unit',
 'Exemption Policy Unit',
 'HealthCare Connections',
 'Land Records',
 'Office of the Sheriff',
 'Property Exec Office',
 'RPIE Unit',
 'Revenue Accounting',
 'Senior Citizen Rent Increase Exemption Unit',
 'TAX',
 'Valuation Policy'}

In [90]:
# Step 5: Loop over all months and combine results
start_date = pd.to_datetime('2018-08-01')
end_date = pd.to_datetime('2022-12-01')
# complaints = pd.DataFrame()
for date in pd.date_range(start=start_date, end=end_date, freq='MS'):
    date_str = date.strftime('%Y-%m')
    print(date_str)
    monthly_complaints = get_complaints_by_month(date_str, nta_nyc)
    complaints = pd.concat([complaints, monthly_complaints])

2018-08
2018-09
2018-10
2018-11
2018-12
2019-01
2019-02
2019-03
2019-04
2019-05
2019-06
2019-07
2019-08
2019-09
2019-10
2019-11
2019-12
2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2020-07
2020-08
2020-09
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
2021-04
2021-05
2021-06
2021-07
2021-08
2021-09
2021-10
2021-11
2021-12
2022-01
2022-02
2022-03
2022-04
2022-05
2022-06
2022-07
2022-08
2022-09
2022-10
2022-11
2022-12


In [91]:
complaints = complaints.fillna(0)
complaints.tail()

,complaint_type_final,adopt-a-basket,air quality,animal abuse,animal facility - no permit,animal in a park,appliance,asbestos,beach/pool/sauna complaint,best/site safety,bike rack condition,...,obstruction,sanitation worker or vehicle complaint,storm,incorrect data,retailer complaint,transfer station complaint,face covering violation,private school vaccine mandate non-compliance,oil or gas spill,building drinking water tank
ntacode,date,,,,,,,,,,,,,,,,,,,,,
SI37,2022-12-01,0.0,4.0,3.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,...,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI45,2022-12-01,1.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI48,2022-12-01,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI54,2022-12-01,0.0,2.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SI99,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
complaints.to_csv('311_monthlycomplaints_nta.csv')

In [93]:
len(complaints)

10329